In [1]:
import bs4
import requests
from lxml import etree
import numpy as np
import spacy
from urllib.request import urlopen
from tqdm import tqdm 
import nltk
from bs4 import Doctype, NavigableString
import json
nltk.download('punkt')

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# HITS

In [ ]:
# url_link = "https://math.illinois.edu/directory/profile/sahlgren"
url_link = "https://mathematics.stanford.edu/people/mohammed-abouzaid"
result = requests.get(url_link).text
soup = bs4.BeautifulSoup(result, "html.parser")

nlp = spacy.load("en_core_web_sm")

In [ ]:
def get_similarities(doc, attributes):
  doc = nlp(doc)
  return [doc.similarity(attribute) for attribute in attributes]

In [ ]:
hub = dict()
auth = dict()
auth_weights = dict()
epsilon = 0

# def dfs(soup, attributes):
#   if soup.name == None:
#     return

#   else:
#     for child in soup.children:
#       hub[child] = 1
#       auth[child] = 1

#       if type(child) == bs4.element.NavigableString:
#         doc = nlp(str(child.encode('utf-8')))
#         auth_weights[child] = max(get_similarities(doc, attributes))
#       else:
#         auth_weights[child] = epsilon

#       dfs(child, attributes)

def dfs(soup, attributes):
  hub[soup] = 1
  auth[soup] = 1
  auth_weights[soup] = 0#epsilon

  if soup.name == None:
    if type(soup) == bs4.element.NavigableString:
      doc = nlp(str(soup.encode('utf-8')))
      auth_weights[soup] = 1 #max(get_similarities(doc, attributes))
    return

  else:
    for child in soup.children:
      dfs(child, attributes)

In [ ]:
attributes = ["research", "publications", "education", "students"]
attributes = [nlp(attribute) for attribute in attributes] 

In [ ]:
dfs(soup, attributes)

In [ ]:
def hub_update():
  for soup in hub:
    if soup.name == None:
      hub[soup] = 0
    
    else:
      hub_score = 0
      for child in soup.children:
        hub_score += auth[child]
      hub[soup] = hub_score

def auth_update():
  for soup in auth:
    parent = soup.parent
    if parent == None:
      auth[soup] = 0
    else:
      auth_score = hub[parent]
      auth[soup] = auth_score * auth_weights[soup]
    
def normalize(d):
  denom = 0
  for key in d:
    denom += d[key]**2
  denom = denom ** (1/2)

  for key in d:
    d[key] /= denom

  return d
  

In [ ]:
for iter in range(100):
  hub_update()
  auth_update()
  hub = normalize(hub)
  auth = normalize(auth)

In [ ]:
A = max(auth, key=auth.get)
print(A)
print(auth[A])
# B = max(hub, key=hub.get)
# print(B)
# print()
# for child in A.children:
#   print(child)
# sorted(hub, key=hub.get, reverse=True)[:5]

Department of Mathematics
0.5773502691896258
